<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Откройте-и-изучите-файл" data-toc-modified-id="Откройте-и-изучите-файл-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Откройте и изучите файл</a></span></li><li><span><a href="#Разбейте-данные-на-выборки" data-toc-modified-id="Разбейте-данные-на-выборки-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Разбейте данные на выборки</a></span></li><li><span><a href="#Исследуйте-модели" data-toc-modified-id="Исследуйте-модели-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Исследуйте модели</a></span></li><li><span><a href="#Проверьте-модель-на-тестовой-выборке" data-toc-modified-id="Проверьте-модель-на-тестовой-выборке-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверьте модель на тестовой выборке</a></span></li><li><span><a href="#(бонус)-Проверьте-модели-на-адекватность" data-toc-modified-id="(бонус)-Проверьте-модели-на-адекватность-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>(бонус) Проверьте модели на адекватность</a></span></li><li><span><a href="#Чек-лист-готовности-проекта" data-toc-modified-id="Чек-лист-готовности-проекта-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>---</a></span></li></ul></div>

# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
from scipy import stats as st
import warnings
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
data_all=pd.read_csv('users_behavior.csv')
#смотрим таблицу
display(data_all.head(10))
print(data_all.info())

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
5,58.0,344.56,21.0,15823.37,0
6,57.0,431.64,20.0,3738.90,1
7,15.0,132.40,6.0,21911.60,0
8,7.0,43.39,3.0,2538.67,1
9,90.0,665.41,38.0,17358.61,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB
None


<i><b>Перед нами датасет, предобработанный, без пропусков и з 3214 строк. Наш целевой признак- колонка 'is_ultra', который говорит о том, пользуется владелец тарифом is_ultra или другим.</i></b>

## Разбейте данные на выборки

In [3]:
train_valid, test = train_test_split(data_all, test_size=0.2)
train, valid = train_test_split(train_valid, test_size=0.2)

features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']
features_valid = valid.drop(['is_ultra'], axis=1)
target_valid = valid['is_ultra']
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

print(data_all.shape)
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)
print(features_test.shape)
print(target_test.shape)


(3214, 5)
(2056, 4)
(2056,)
(515, 4)
(515,)
(643, 4)
(643,)


## Исследуйте модели

<i><b>Классификатор дерева решений</i></b>

In [4]:
best_model = None
best_result = 0
best_depth=0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    model.fit(features_train,target_train) 
    predictions_valid  = model.predict(features_valid)
    result=accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
        best_depth = depth
    
print("Accuracy")
print(best_result,'max_depth=',best_depth)    

Accuracy
0.7941747572815534 max_depth= 4


<i><b>Случайный лес</i></b>

In [5]:
best_model = None
best_result = 0
best_est = 0
for est in range(1, 110):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train,target_train)
    result = model.score(features_valid,target_valid) 
    if result > best_result:
        best_model = model
        best_result = result
        best_est = est

print("Accuracy наилучшей модели на валидационной выборке:", best_result,'est=',best_est)

Accuracy наилучшей модели на валидационной выборке: 0.7747572815533981 est= 52


<i><b>Логистическая регрессия</i></b>

In [6]:
model =LogisticRegression(random_state=12345)  
model.fit(features_train,target_train)
result = model.score(features_valid,target_valid) 

print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.7165048543689321


<i><b>Лучшая модель по accuracy - Это случайный лес</i></b>

## Проверьте модель на тестовой выборке

<i>Проверим работу модели "Классификатор дерева решений" на тестовой выборке</i>

In [9]:
model = RandomForestClassifier(random_state=12345, n_estimators=best_est)
model.fit(features_train,target_train) 
result=model.score(features_valid, target_valid)   
result_test=model.score(features_test,target_test) 
    

print("Accuracy наилучшей модели на валидационной выборке:", result,'n_estimators=',best_est)
print("Accuracy наилучшей модели на тестовой выборке:", result_test,'n_estimators=',best_est)


Accuracy наилучшей модели на валидационной выборке: 0.7747572815533981 n_estimators= 52
Accuracy наилучшей модели на тестовой выборке: 0.7931570762052877 n_estimators= 52


<i>Видим что тестовая выборка показывает немного худший результат чем валидационная.</i>

## (бонус) Проверьте модели на адекватность

In [8]:
#Возьмем стратегию самое популярное число.
print(data_all['is_ultra'].value_counts())
#У нас это 0.
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(features_train, target_train)
predictions_dummy_valid=dummy_clf.predict(features_valid)
dummy_result_valid=accuracy_score(target_valid, predictions_dummy_valid)
predictions_dummy_test  = dummy_clf.predict(features_test)
dummy_result_test=accuracy_score(target_test, predictions_dummy_test)
print("Accuracy dummy модели на валидационной выборке:", dummy_result_valid)
print("Accuracy dummy модели на тестовой выборке:", dummy_result_test)


0    2229
1     985
Name: is_ultra, dtype: int64
Accuracy dummy модели на валидационной выборке: 0.7009708737864078
Accuracy dummy модели на тестовой выборке: 0.7013996889580093


<i>Видим что модель "Классификатор дерева решений" точнее dummy модели "самое частое значение"</i>